In [3]:
import os
import sys
from time import time
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import networkx as nx
import random

from keras import layers, models, optimizers
from keras import backend as K
K.set_image_data_format('channels_last')
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks


from utils import combine_images
from utils import plot_log

from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
sys.path.append('./PatchyTools/')

from PatchyConverter import PatchyConverter
from DropboxLoader import DropboxLoader
from CapsuleParameters import CapsuleParameters
from CapsuleParameters import CapsuleTrainingParameters
from GraphClassifier import GraphClassifier


import matplotlib.pyplot as plt
%matplotlib inline

### Load data

In [2]:
# Getting the data:
dataset_name = 'MUTAG'
width = 18
receptive_field = 10
relabeling = True

In [4]:
graph_converter = PatchyConverter(dataset_name,receptive_field)

importing graph data
number of graphs in MUTAG dataset : 188
number of features : 7
Separating Graphs per graph ID
getting width
width: 18


In [6]:
graph_converter.create_adj_coomatrix_by_graphId()

In [8]:
graph_converter.canonical_labeling()

In [17]:
graph_converter.node_label_by_graphId[1]

,index,label,node,graph_ind,cano_label
0,0,0,0,1,15
1,1,0,1,1,16
2,2,0,2,1,13
3,3,0,3,1,11
4,4,0,4,1,7
5,5,0,5,1,6
6,6,0,6,1,2
7,7,0,7,1,1
8,8,0,8,1,0
9,9,0,9,1,5


In [18]:
from PatchyConverter import dfadj_to_dict

In [19]:
temp_graph_dict = dfadj_to_dict(graph_converter.adj_dict_by_graphId[1])
nauty_graph = nauty.Graph(len(temp_graph_dict), adjacency_dict=temp_graph_dict)

In [30]:
nauty.autgrp?

In [11]:
import pynauty as nauty

In [27]:
nauty.canonical_labeling?

In [4]:
# Adjeceny dict and 
PatchyConverter = GraphConverter(dataset_name, receptive_field)
adj_dict = PatchyConverter.create_adj_dict_by_graphId()
adj_coomatrix_by_graphId = PatchyConverter.create_adj_coomatrix_by_graphId(adj_dict)
#cano_label = canonical_labeling(adj_dict, df_node_label, df_adj)

In [ ]:
# Importing graph features:
#df_edge_label = dropbox_loader.get_edge_label()
df_graph_ind = dropbox_loader.get_graph_ind()
df_adj = dropbox_loader.get_adj()
df_node_label = dropbox_loader.get_node_label()
df_node_label = pd.concat([df_node_label, df_graph_ind.graph_ind], axis=1)

In [ ]:
# Converting Graphs into Matrices:
if relabeling:
    PatchyConverter.relabel_graphs()
graph_tensor = PatchyConverter.graphs_to_Patchy_tensor()

In [ ]:
# Capsule Architecture Parameters:
capsule_params= CapsuleParameters()

# First conv layer: 'filters', kernel_size)
conv_layer_name = 'conv_layer'
conv_layer_params = {}
conv_layer_params['filters'] = 256
conv_layer_params['kernel_size'] = 9
conv_layer_params['strides'] = [1, 1]
conv_layer_params['padding'] = 'VALID'
conv_layer_params['activation'] = 'relu'
conv_layer_params['name'] = 'conv1'

capsule_params.add_params(conv_layer_params,conv_layer_name)

# First Capsule Layer:
# [num_output_caps, caps_len,'filters',kernel_size,strides,padding]
caps_layer_name = 'caps_layer'
caps_layer_params = {}
caps_layer_params['filters'] = 256
caps_layer_params['kernel_size'] = 2
caps_layer_params['strides'] = [2,2]
caps_layer_params['padding'] = 'VALID'
caps_layer_params['padding'] = 'VALID'
caps_layer_params['n_channels'] = 32
caps_layer_params['dim_capsule'] = 8
caps_layer_params['name'] = 'caps_layer'
capsule_params.add_params(caps_layer_params,caps_layer_name)

# Digit Capsule Layer:
digit_layer_name = 'digitcaps_layer'
digit_layer_params = {}
digit_layer_params['n_channels'] = 10
digit_layer_params['dim_capsule'] = 16
digit_layer_params['name'] = 'digitcaps'
capsule_params.add_params(digit_layer_params,digit_layer_name )

# Capsule Decoder:
decoder_layer = 'decoder_layer'
decoder_params = {}
decoder_params['first_dense'] = 256 #250 #512
decoder_params['second_dense'] = 512
decoder_params['name'] = 'decoder'
capsule_params.add_params(decoder_params,decoder_layer)


In [ ]:
parameter_set = CapsuleTrainingParameters()
parameter_set.add_fold(1)
parameter_set.epochs= 5

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(graph_tensor,graph_labels,test_size=0.10,random_state=1)
data = ((x_train, y_train), (x_test, y_test))
input_shape = x_train.shape[1:]

In [ ]:
patchy_classifier = GraphClassifier(input_shape)
patchy_classifier.build_the_graph(capsule_params)
patchy_classifier.train(data, parameter_set)

In [ ]:
patchy_classifier.results.rename('wegewg',inplace=True)

In [ ]:
patchy_classifier.results.name